## Load Data

In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta

sp = pd.read_csv('SP500.csv')
dj = pd.read_csv('DJIA.csv')
shcomp = pd.read_csv('SHCOMP.csv')
dax = pd.read_csv('DAX.csv')
nikkei = pd.read_csv('NIKKEI225.csv')
sp.columns

Index(['Date', ' Open', ' High', ' Low', ' Close'], dtype='object')

In [2]:
def prepare_data(df):
    # date formatting
    df['Date'] = pd.to_datetime(df['Date'])
    df['Date'] = np.where(df['Date'].dt.year > 2050, df['Date'] - pd.offsets.DateOffset(years=100), df['Date'])
    df['Date'] = df['Date'].dt.date
    
    # remove spaces in column names
    df.columns = df.columns.str.replace(' ', '')
    return df.sort_values(by='Date', ascending=True).reset_index(drop=True)

In [3]:
sp = prepare_data(sp)
dj = prepare_data(dj)
shcomp = prepare_data(shcomp)
dax = prepare_data(dax)
nikkei = prepare_data(nikkei)
nikkei

,Date,Open,High,Low,Close
0,1960-03-18,1021.33,1021.33,1021.33,1021.33
1,1960-03-21,1024.06,1024.06,1024.06,1024.06
2,1960-03-22,1033.06,1033.06,1033.06,1033.06
3,1960-03-23,1033.30,1033.30,1033.30,1033.30
4,1960-03-24,1039.52,1039.52,1039.52,1039.52
...,...,...,...,...,...
15663,2022-06-10,27996.35,28044.45,27795.17,27824.29
15664,2022-06-13,27369.66,27389.30,26948.22,26987.44
15665,2022-06-14,26555.75,26657.92,26357.90,26629.86
15666,2022-06-15,26625.68,26638.76,26321.68,26326.16


## Trading Bot Setup

In [4]:
money_format = "${:,.2f}"
share_format = "{:,.2f}"
ror_format = "{:,.2f}%"
def fmt_money(money):
    return money_format.format(money)
def fmt_shares(shares):
    return share_format.format(shares)
def fmt_ror(ror):
    return ror_format.format(100 * (ror - 1))

class TradingBot:
    def __init__(self, training_data, test_data, money=1000):
        self.train_data = training_data
        self.test_data = test_data
        self.money = money
        self.start_money = money
        self.shares = 0
    
    def execute(self, print_frequency=1265):
        self.train(print_frequency=print_frequency)
        
        # self.summary(self.test_data.iloc[0])
        self.test(print_frequency=print_frequency)
        print('-' * 50)
        self.summary(self.test_data.iloc[-1])
        
    def train(self, print_frequency=1265):
        raise NotImplementedError()
    
    def test(self, print_frequency=1265):
        raise NotImplementedError()

    def summary(self, row):
        share_worth = self.shares * row['Close']
        total_worth = self.total_worth(row)
        
        print(f'{row.Date}')
        print(f'{fmt_money(self.money)} in cash, {fmt_money(share_worth)} in shares ({fmt_shares(self.shares)} shares)')
        print(f'= {fmt_money(total_worth)} net worth at {fmt_money(row.Close)}/share')
        
        t0 = self.test_data.iloc[0]['Date']
        tend = row['Date']
        year_diff = (tend - t0) / timedelta(days=365)
        
        if year_diff > 0:
            P0 = self.start_money
            Pend = total_worth
            ror = (Pend / P0)**(1 / year_diff)
            print(f'= {fmt_ror(ror)} average annual rate of return (since {t0})')
        
        print()
        
    # positive numbers to buy, negative numbers to sell
    def trade(self, price, amount_money=None, amount_shares=None, debug_output=False):
        if (amount_money is None and amount_shares is None) or \
            (amount_money is not None and amount_shares is not None):
                raise ValueError()
                
        upper = self.money
        lower = -price * self.shares
        
        # set the other of amount_money if not specified
        if amount_shares is not None:
            amount_money = price * amount_shares

        amount_money = max(min(amount_money, upper), lower)
        amount_shares = amount_money/price
        self.money -= amount_money
        self.shares += amount_shares
        
        if debug_output:
            action = 'Bought' if amount_money >= 0 else 'Sold'
            print(f'{action} {fmt_shares(abs(amount_shares))} shares for {fmt_money(abs(amount_money))}')
            print(f'\tat {fmt_money(price)}/share.')
            
    def total_worth(self, row):
        return self.shares * row['Close'] + self.money
        
class DailyTrader(TradingBot):
    def __init__(self, data, n_day_window, money=1000):
        super().__init__(None, data, money=money)
        self.window = n_day_window
        
    def train(self, print_frequency=1265):
        pass
    
    # about 253 trading days a year
    def test(self, print_frequency=1265):
        for index, row in self.test_data.iterrows():
            if index < self.window:
                continue
            
            window_data = self.test_data.iloc[index - self.window:index + 1].reset_index(drop=True)
            self.daily_trade(window_data)
            
            output = (index % print_frequency == 0)
            if output:
                self.summary(row)
    
    def daily_trade(self, window_data):
        raise NotImplementedError()

In [5]:
class HoldTrader(TradingBot):
    def __init__(self, data, money=1000):
        super().__init__(None, data, money=money)
        
    def train(self, print_frequency=1265):
        pass
    
    def test(self, print_frequency=1265):
        price = self.test_data.iloc[0].Open
        self.trade(price, amount_money=self.money)
        
        for index, row in self.test_data.iterrows():
            output = (index % print_frequency == 0)
            if output:
                self.summary(row)

In [6]:
def is_monotonic_inc(col):
    return col.is_monotonic_increasing and col.is_unique

def is_monotonic_dec(col):
    return col.is_monotonic_decreasing and col.is_unique

class UpTrader(DailyTrader):
    def __init__(self, data, money=1000):
        super().__init__(data, 1, money=money)
        
    def daily_trade(self, window_data):
        prev_price = window_data.iloc[0].Open
        cur_price = window_data.iloc[1].Open
        
        if cur_price > prev_price:
            self.trade(cur_price, amount_money=self.money)
        else:
            self.trade(cur_price, amount_shares=-self.shares)
            
class DownTrader(DailyTrader):
    def __init__(self, data, money=1000):
        super().__init__(data, 1, money=money)
        
    def daily_trade(self, window_data):
        prev_price = window_data.iloc[0].Open
        cur_price = window_data.iloc[1].Open
        
        if cur_price < prev_price:
            self.trade(cur_price, amount_money=self.money)
        else:
            self.trade(cur_price, amount_shares=-self.shares)
            
class CustomTrader(DailyTrader):
    def __init__(self, data, money=1000):
        super().__init__(data, 180, money=money)
        
        row = self.test_data.iloc[0]
        self.trade(row.Open, amount_money=self.money)
        
    def daily_trade(self, window_data):
        # get the minimum over the last 20 days
        mn = window_data.iloc[-21:-1].Open.min()
        
        # get the maximum over the last 180 days
        mx = window_data.iloc[:-1].Open.max()
        
        prev_price = window_data.iloc[-2].Open
        cur_price = window_data.iloc[-1].Open
        
        # sell if the current_price is the maximum over k days
        if cur_price > prev_price * 1.07:
            self.trade(cur_price, amount_shares=-self.shares * 1, debug_output=False)

        # buy if the current price is the minimum over k days
        elif cur_price < mn:
            self.trade(cur_price, amount_money=self.money * .3, debug_output=False)
        else:
            self.trade(cur_price, amount_money=self.money * .01, debug_output=False)
            
class CustomTrader2(DailyTrader):
    def __init__(self, data, money=1000):
        super().__init__(data, 180, money=money)
        
        row = self.test_data.iloc[0]
        self.trade(row.Open, amount_money=self.money)
        
    def daily_trade(self, window_data):
        # check if the last 20 days are increasing
        if is_monotonic_inc(window_data.iloc[-21:-1].Open):
            self.trade(window_data.iloc[-1].Open, amount_money=self.money)
        else:
            self.trade(window_data.iloc[-1].Open, amount_shares=-self.shares * .01)

In [7]:
def compare_bots(data, *args):
    for cls in args:
        print(f'Trading Bot: {cls.__name__}')
        print('_' * 50)
        print()
        bot = cls(data, money=1000)
        bot.execute()
        print()
        
compare_bots(sp, CustomTrader, CustomTrader2, HoldTrader)

Trading Bot: CustomTrader
__________________________________________________

1983-01-05
$0.00 in cash, $1,513.11 in shares (10.66 shares)
= $1,513.11 net worth at $141.96/share
= 8.62% average annual rate of return (since 1978-01-03)

1988-01-07
$571.50 in cash, $2,443.61 in shares (9.36 shares)
= $3,015.11 net worth at $261.07/share
= 11.65% average annual rate of return (since 1978-01-03)

1993-01-07
$0.00 in cash, $5,004.40 in shares (11.62 shares)
= $5,004.40 net worth at $430.73/share
= 11.32% average annual rate of return (since 1978-01-03)

1998-01-08
$0.00 in cash, $11,107.78 in shares (11.62 shares)
= $11,107.78 net worth at $956.05/share
= 12.77% average annual rate of return (since 1978-01-03)

2003-01-22
$0.00 in cash, $10,205.15 in shares (11.62 shares)
= $10,205.15 net worth at $878.36/share
= 9.71% average annual rate of return (since 1978-01-03)

2008-01-31
$0.00 in cash, $16,016.56 in shares (11.62 shares)
= $16,016.56 net worth at $1,378.55/share
= 9.65% average annu

In [8]:
compare_bots(sp, UpTrader)

Trading Bot: UpTrader
__________________________________________________

1983-01-05
$0.00 in cash, $2,410.10 in shares (16.98 shares)
= $2,410.10 net worth at $141.96/share
= 19.20% average annual rate of return (since 1978-01-03)

1988-01-07
$0.00 in cash, $5,180.21 in shares (19.84 shares)
= $5,180.21 net worth at $261.07/share
= 17.85% average annual rate of return (since 1978-01-03)

1993-01-07
$7,655.30 in cash, $0.00 in shares (0.00 shares)
= $7,655.30 net worth at $430.73/share
= 14.51% average annual rate of return (since 1978-01-03)

1998-01-08
$15,811.99 in cash, $0.00 in shares (0.00 shares)
= $15,811.99 net worth at $956.05/share
= 14.78% average annual rate of return (since 1978-01-03)

2003-01-22
$16,097.94 in cash, $0.00 in shares (0.00 shares)
= $16,097.94 net worth at $878.36/share
= 11.72% average annual rate of return (since 1978-01-03)

2008-01-31
$12,294.42 in cash, $0.00 in shares (0.00 shares)
= $12,294.42 net worth at $1,378.55/share
= 8.69% average annual rate